In [2]:
import sqlite3
import hashlib
def setup_database():
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        username TEXT UNIQUE,
                        password TEXT)''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS transactions (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        user_id INTEGER,
                        amount REAL,
                        category TEXT,
                        type TEXT,
                        FOREIGN KEY(user_id) REFERENCES users(id))''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS budgets (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        user_id INTEGER,
                        category TEXT,
                        amount REAL,
                        FOREIGN KEY(user_id) REFERENCES users(id))''')
    conn.commit()
    conn.close()

setup_database()

In [3]:
def create_user(username, password):
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    try:
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        print("User  registered successfully!")
    except sqlite3.IntegrityError:
        print("Username already exists.")
    conn.close()

def login(username, password):
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    cursor.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, hashed_password))
    user = cursor.fetchone()
    conn.close()
    return user is not None

In [4]:
def add_transaction(user_id, amount, category, transaction_type):
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO transactions (user_id, amount, category, type) VALUES (?, ?, ?, ?)", 
                   (user_id, amount, category, transaction_type))
    conn.commit()
    conn.close()

def view_transactions(user_id):
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM transactions WHERE user_id = ?", (user_id,))
    transactions = cursor.fetchall()
    conn.close()
    return transactions

In [5]:
def generate_report(user_id, period='monthly'):
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    if period == 'monthly':
        cursor.execute("SELECT SUM(amount) FROM transactions WHERE user_id = ? AND type = 'income'", (user_id,))
        total_income = cursor.fetchone()[0] or 0
        cursor.execute("SELECT SUM(amount) FROM transactions WHERE user_id = ? AND type = 'expense'", (user_id,))
        total_expenses = cursor.fetchone()[0] or 0
    conn.close()
    return total_income, total_expenses, total_income - total_expenses

In [6]:
def set_budget(user_id, category, budget_amount):
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO budgets (user_id, category, amount) VALUES (?, ?, ?)", 
                   (user_id, category, budget_amount))
    conn.commit()
    conn.close()

def check_budget(user_id, category):
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    cursor.execute("SELECT amount FROM budgets WHERE user_id = ? AND category = ?", (user_id, category))
    budget = cursor.fetchone()
    cursor.execute("SELECT SUM(amount) FROM transactions WHERE user_id = ? AND category = ?", (user_id, category))
    spent = cursor.fetchone()[0] or 0
    conn.close()
    return budget[0] if budget else 0, spent

In [7]:
def main_menu():
    while True:
        print("\n--- Personal Finance Management Application ---")
        print("1. Register")
        print("2. Login")
        print("3. Exit")
        choice = input("Select an option: ")

        if choice == '1':
            username = input("Enter a username: ")
            password = input("Enter a password: ")
            create_user(username, password)
        elif choice == '2':
            username = input("Enter your username: ")
            password = input("Enter your password: ")
            if login(username, password):
                print("Login successful!")
                user_dashboard(username)
            else:
                print("Invalid username or password.")
        elif choice == '3':
            print("Exiting the application.")
            break
        else:
            print("Invalid choice. Please try again.")

In [8]:
def user_dashboard(username):
    conn = sqlite3.connect('finance_app.db')
    cursor = conn.cursor()
    cursor.execute("SELECT id FROM users WHERE username = ?", (username,))
    user_id = cursor.fetchone()[0]
    conn.close()

    while True:
        print("\n--- User Dashboard ---")
        print("1. Add Transaction")
        print("2. View Transactions")
        print("3. Generate Financial Report")
        print("4. Set Budget")
        print("5. Check Budget")
        print("6. Logout")
        choice = input("Select an option: ")

        if choice == '1':
            amount = float(input("Enter the transaction amount: "))
            category = input("Enter the transaction category (e.g., Food, Rent, Salary): ")
            transaction_type = input("Enter the transaction type (income/expense): ").lower()
            add_transaction(user_id, amount, category, transaction_type)
            print("Transaction added successfully.")
        elif choice == '2':
            transactions = view_transactions(user_id)
            print("\n--- Transactions ---")
            for t in transactions:
                print(t)
        elif choice == '3':
            total_income, total_expenses, savings = generate_report(user_id)
            print(f"Total Income: {total_income}, Total Expenses: {total_expenses}, Savings: {savings}")
        elif choice == '4':
            category = input("Enter the budget category: ")
            budget_amount = float(input("Enter the budget amount: "))
            set_budget(user_id, category, budget_amount)
            print("Budget set successfully.")
        elif choice == '5':
            category = input("Enter the budget category to check: ")
            budget, spent = check_budget(user_id, category)
            print(f"Budget for {category}: {budget}, Amount spent: {spent}")
        elif choice == '6':
            print("Logging out...")
            break
        else:
            print("Invalid choice. Please try again.")

In [ ]:
main_menu()


--- Personal Finance Management Application ---
1. Register
2. Login
3. Exit
Select an option: 1
Enter a username: swati
Enter a password: swati@
User  registered successfully!

--- Personal Finance Management Application ---
1. Register
2. Login
3. Exit
Select an option: swati
Invalid choice. Please try again.

--- Personal Finance Management Application ---
1. Register
2. Login
3. Exit
Select an option: 2
Enter your username: swati
Enter your password: swati@
Login successful!

--- User Dashboard ---
1. Add Transaction
2. View Transactions
3. Generate Financial Report
4. Set Budget
5. Check Budget
6. Logout
Select an option: 1
Enter the transaction amount: 700
Enter the transaction category (e.g., Food, Rent, Salary): salary
Enter the transaction type (income/expense): income
Transaction added successfully.

--- User Dashboard ---
1. Add Transaction
2. View Transactions
3. Generate Financial Report
4. Set Budget
5. Check Budget
6. Logout
Select an option: 2

--- Transactions ---
(2,